# 構造データの充填率 filling rateを求める
今回は構造データの0(空孔)と1(固体)の割合(充填率 : filling_rate)と目的変数(m, kappa)の相関関係を見ていくための準備である。

In [4]:
# --- ライブラリのインポート ---
import os
import numpy as np
import h5py
import scipy.io as sio

In [5]:
# --- 定数の設定 ---
# .matのデータフォルダ場所
MAT_PATH = "/mnt/c/Users/onion/Documents/data"

# npyのデータフォルダ保存場所
NPY_PATH = f"/mnt/c/Users/onion/Documents/analysis"

# pathのディレクトリ部分が存在しない場合は作成、存在してもエラーにしない
os.makedirs(NPY_PATH, exist_ok=True)
print(f"{NPY_PATH} が作成されました。")

# targetのデータ名
TARGET_NAMES = ["m_train", "m_val", "m_test", "kappa_train", "kappa_val", "kappa_test"]

# 保存データ名
SAVE_NAMES = ["FR_train", "FR_val", "FR_test"]

# .matのstructureデータフォルダ場所
STRUCTURE_MAT_PATHS = [os.path.join(MAT_PATH, "structures_train"),
                      os.path.join(MAT_PATH, "structures_val"),
                      os.path.join(MAT_PATH, "structures_test")]

# .matのstructureファイル数
STRUCTURE_MAT_NUMS = [len([f for f in os.listdir(path) if f.endswith(".mat")]) for path in STRUCTURE_MAT_PATHS]
print(STRUCTURE_MAT_NUMS)


/mnt/c/Users/onion/Documents/analysis が作成されました。
[63000, 13500, 13500]


In [6]:
# --- keyの取得 ---
# targetのkeysを取得
for TARGET_NAME in TARGET_NAMES:
    target_mat_path = os.path.join(MAT_PATH, f"{TARGET_NAME}.mat")

    with h5py.File(target_mat_path, "r") as f:
        keys = list(f.keys())
        print(f"{TARGET_NAME}.mat のkeyは {keys} が設定されているよ")

# 初めの3つのstructureのkeysを取得
for i in range(3):
    train_structure_mat_path = os.path.join(STRUCTURE_MAT_PATHS[0], f"structure_{i+1}.mat")
    data = sio.loadmat(train_structure_mat_path)
    keys = list(data.keys())
    print(f"structure_{i}.mat のkeyは {keys} が設定されているよ")

m_train.mat のkeyは ['y'] が設定されているよ
m_val.mat のkeyは ['y'] が設定されているよ
m_test.mat のkeyは ['y'] が設定されているよ
kappa_train.mat のkeyは ['y'] が設定されているよ
kappa_val.mat のkeyは ['y'] が設定されているよ
kappa_test.mat のkeyは ['y'] が設定されているよ
structure_0.mat のkeyは ['__header__', '__version__', '__globals__', 'BW'] が設定されているよ
structure_1.mat のkeyは ['__header__', '__version__', '__globals__', 'BW'] が設定されているよ
structure_2.mat のkeyは ['__header__', '__version__', '__globals__', 'BW'] が設定されているよ


In [7]:
# --- keyの設定 ---
# 上記のコードから取得したキーからほしいデータだけを指定する
# ただし、キーがすべて同じである前提である
TARGET_KEY = 'y'
STRUCTURE_KEY = 'BW'

In [5]:
# --- targetのmat2npy ---
for TARGET_NAME in TARGET_NAMES:
    target_mat_path = os.path.join(MAT_PATH, f"{TARGET_NAME}.mat")
    target_npy_path = os.path.join(NPY_PATH, f"{TARGET_NAME}.npy")

    # MATLABファイルを読み込み、指定された変数をPyTorchのテンソルに変換して保存
    with h5py.File(target_mat_path, "r") as f:
        data = f[TARGET_KEY]
        data = np.array(data).T
        np.save(target_npy_path, data)

        if os.path.exists(target_npy_path):
            print(f"{TARGET_NAME}.npy が作成されました。")

m_train.npy が作成されました。
m_val.npy が作成されました。
m_test.npy が作成されました。
kappa_train.npy が作成されました。
kappa_val.npy が作成されました。
kappa_test.npy が作成されました。


In [8]:
# --- structureのmat2npy (Filling rate) ---
for i in range(1, 3):# range(len(STRUCTURE_MAT_PATHS)):

    filling_rate = []

    for j in range(STRUCTURE_MAT_NUMS[i]):
        structure_mat_path = os.path.join(STRUCTURE_MAT_PATHS[i], f"structure_{j+1}.mat")
        FR_npy_path = os.path.join(NPY_PATH, f"{SAVE_NAMES[i]}.npy")

        data = sio.loadmat(structure_mat_path)
        data = data[STRUCTURE_KEY]
        data = np.array(data)

        filling_rate.append(np.sum(data) / data.size)

    np.save(FR_npy_path, filling_rate)